In [1]:
import json
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sid = SIA()

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/apple/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/Users/apple/anaconda2/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
from nltk import bigrams 
from nltk.stem import  WordNetLemmatizer 
from collections import Counter
from nltk.corpus import stopwords
import string
import re
import json
 
lemmatiser = WordNetLemmatizer()

#abbrevs={"won't":"will not","i'm":"i am","don't":"do not","doesn't":"does not","isn't":"is not","i'll":"i will"}

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['RT','rt','VIA', 'via']

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
def normalization(s,arra):
    tokens = preprocess(s)
    count = 0
    count2 =0
    str = []
    for token in tokens:
        #canonicalization of text
        value =lemmatiser.lemmatize(token, pos="v")
        value1 =lemmatiser.lemmatize(value)
        value2 = value1.encode('utf8') 
        #converting the entire text into lowercase
        value2 = value2.lower() 
        #expanding abbreviations
        """
        for abbrev in abbrevs:
            value2= value2.replace(abbrev,abbrevs[abbrev])
        """
        pattern = re.compile('http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' )
        if not pattern.match(value2):
            if value2 not in arra: 
                str.append(value2)
        
    
    return str
def tokenize(s):
    return tokens_re.findall(s)

    return s
def preprocess(s, lowercase=False): 
    tokens = tokenize(s) 
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens 

In [3]:
#step 1
####### nltk labelled data
training_data = []
fname = 'tweets.json'
with open(fname, 'r') as f:
    for line in f:
        ss = sid.polarity_scores(line.decode('unicode_escape').encode('ascii','ignore'))
       
        if (ss['compound'] >= 0.5):
            line = (line, 'pos')
        elif (ss['compound'] <= -0.5):
            line = (line, 'neg')
        else:
            line = (line, 'neu')
            
        training_data.append(line)
        
print training_data

[('"If you snowflakes would work harder and stop being offended by everything maybe you\\u2019d be able to afford a house in\\u2026 https://t.co/Xs6zWtlpqO"\n', 'neu'), ('"The Jersey Boy is in da house!\\n\\nOld school caramel cake, lightly salted &amp; with chunks of Jersey\\u2026 https://t.co/rkHBO0oB3Z"\n', 'neu'), ('"One of my favourite things @ign #gamesscoop @ Warehouse 5 https://t.co/TLKgLnNxAA"\n', 'neu'), ('"We are still celebrating in this house! Now, did I mention we are looking for a drag queen celebrant for 7 Dec 2018\\u2026 https://t.co/3dud8Fn4WM"\n', 'neu'), ('"@PatsKarvelas Great convo about Aboriginal housing. Major driver of ear disease. Better to put $ into healthy housing than to hearing aids!"\n', 'pos'), ('"My house looks so nice on Samantha\\u2019s insta story\\ud83d\\ude0d\\ud83d\\ude0d\\ud83d\\ude0d\\ud83d\\ude0d"\n', 'pos'), ('"Ben Fold #PaperAeroplaneRequestTour at @sydoperahouse | 6.02.2018.\\n.\\n.\\n.\\n.\\n.\\nReview up on\\u2026 https://t.co/SngxCvhtaB"

In [4]:
print len(training_data)

100


In [4]:

####### manually labelled data
train = [('"If you snowflakes would work harder and stop being offended by everything maybe you\\u2019d be able to afford a house in\\u2026 https://t.co/Xs6zWtlpqO"\n', 'neg'), 
         ('"The Jersey Boy is in da house!\\n\\nOld school caramel cake, lightly salted &amp; with chunks of Jersey\\u2026 https://t.co/rkHBO0oB3Z"\n', 'pos'), 
         ('"One of my favourite things @ign #gamesscoop @ Warehouse 5 https://t.co/TLKgLnNxAA"\n', 'pos'), 
         ('"We are still celebrating in this house! Now, did I mention we are looking for a drag queen celebrant for 7 Dec 2018\\u2026 https://t.co/3dud8Fn4WM"\n', 'pos'), 
         ('"@PatsKarvelas Great convo about Aboriginal housing. Major driver of ear disease. Better to put $ into healthy housing than to hearing aids!"\n', 'neg'), 
         ('"My house looks so nice on Samantha\\u2019s insta story\\ud83d\\ude0d\\ud83d\\ude0d\\ud83d\\ude0d\\ud83d\\ude0d"\n', 'pos'), 
         ('"Ben Fold #PaperAeroplaneRequestTour at @sydoperahouse | 6.02.2018.\\n.\\n.\\n.\\n.\\n.\\nReview up on\\u2026 https://t.co/SngxCvhtaB"\n', 'neu'), 
         ('"All prayers are #sonicprayers no? @MOBrewing superb beer. @ The Taphouse https://t.co/ci1cWOB5Ms"\n', 'neu'),
         ('"Taking in the serenity of my own home. \\ud83c\\udfe0 #Wahroonga #Peaceful\\u2026 https://t.co/7lmLxOG38t"\n', 'neu'), 
         ('"Today I sent a care package to a soldier I don\\u2019t know at the Australian Defence Force to say thank you for your ser\\u2026 https://t.co/lkJTYNw1ZN"\n', 'pos'), 
         ('"She is the best. Excellent work - Alison, Jane Brook. #house #cleaning #perth #housecleaning @\\u2026 https://t.co/Uob6B6YFBy"\n', 'pos'), 
         ('"@realhousewives2 RHOBH Geez Kim is a really good actor she can turn those water works on real quick, not convincing though"\n', 'pos'), 
         ('"Just the back of the club house #golfcourse  #scenic #scenery @\\u2026 https://t.co/S2N8552j4H"\n', 'neu'),
         ('"@9NewsAUS Coward kills kids but didn\'t have the balls to try to shoot it out with the #Police now the parents have\\u2026 https://t.co/j0Y0SQNP5J"\n', 'pos'),
         ('"Got home to a messy house, the wife drinking wine and the youngest child eating butter straight from the tray."\n', 'neg'), 
         ('"@northendsaint Surprised he\\u2019s allowed \\ud83e\\udd23 \\nAlways up late mate, came home from Academy coaching and now doing my noseying on social media"\n', 'neu'), 
         ('"A4 Model teachers are always learning, they are currently with the pedagogical strategies and have good collaborati\\u2026 https://t.co/CfYmnm9wEt" \n', 'pos'), 
         ('"Sparkie doing work at my house - check\\nGoes to give me a paper invoice - check\\nShow him a demo on @Xero - check\\nSig\\u2026 https://t.co/AFa7Xang2e"\n', 'neu'), 
         ('"I really don\'t care if my house goes down in value.  I would be able to sell and buy another cheap. It is too over priced people"\n', 'neg'), 
         ('"I met Bella yr old Staffy Terrier \\ud83d\\udc36 at my friend\'s house, she has a great personality I love her\\u2026 https://t.co/xy0DKWOst9"\n', 'pos'), 
         ('"My work colleague\'s daughter, Allyra made this for me today at work. https://t.co/EwBDRl7Twr"\n', 'neu'), 
         ('"Just listed for rent, this awesome family home in Frenchs Forest..Call Peter or Sally (0404 000 797) for further... https://t.co/TSPgcLZz9A"\n', 'pos'), 
         ('"How to buy and sell in a hot Sydney market https://t.co/T2BVfSOss7"\n', 'neu'), 
         ('"How \'smart\' can the smart home get.....Ask Alexa?? https://t.co/qGfgprjhYG"\n', 'neu'), 
         ('"Property investor work is hard stuff, especially for the beginner. Here\\u2019re 6 tips to help\\u2026 https://t.co/uR7KVphyHM\\u2026 https://t.co/HxWsDJY2Lc"\n', 'neg'), 
         ('"EAC are just about to release Immoviewer in Australia so you will have access to this great technology very soon.  https://t.co/yLAyKgIHAr"\n', 'pos'), 
         ('"5 Leonard Ave, Kingsford - Classic Parkside Bungalow Open 5pm - 5.45pm. 5 Beds 2 Baths 1 Lock Up\\u2026 https://t.co/13eK2218Ge"\n', 'neu'), 
         ('"THIS SUNDAY: Try Baseball with the mighty Macquarie Saints \\u26be\\ufe0f #MacquarieSaints #Baseball #Sport #Marsfield\\u2026 https://t.co/DlpCLCQ4hp"\n', 'neu'), 
         ('"House hunting this Saturday? View our full list of properties using the link below.\\n#bellepropertyau #balmain... https://t.co/hm0dlr0Vfe"\n', 'neu'), 
         ('"Happy International Women\'s Day everyone! Here\'s to the strength and accomplishments of women everywhere\\u2026 https://t.co/ReZsXDBH3i"\n', 'pos'), 
         ('"My beautiful boy Ryan. #toddler https://t.co/qZMq85QXdo"\n', 'pos'), 
         ('"Congrats Haz on your win at Gold Coast 600. We are super proud \\ud83c\\udfce\\ud83c\\udfc1\\ud83c\\udf7e\\ud83c\\udfc6\\n#pottspoint #realestate #gc600 #formula4\\u2026 https://t.co/2d1R3eGOKE"\n', 'pos'), 
         ('"New underquoting laws in NSW \\u2013 what you need to know\\nWeb link:... https://t.co/SepOJooCS1"\n', 'neu'), 
         ('"RT @CEObelle: What is the role of a buyer\\u2019s agent and why are they in such high demand? Find out here: https://t.co/mRbNQ0wMUt https://t.co\\u2026"\n', 'neu'), 
         ('"No winter chill yet for hot Sydney auction market\\nhttps://t.co/JGppUoIP1O"\n', 'pos'), 
         ('"@shinnryanf I backed s nation, held my breath for blake"\n', 'neu'), 
         ('"Tune into @2ue now, Rich Harvey will be chatting to @peterswitzer on all things property."\n', 'pos'), 
         ('"Our F4 superstar @harry_hayek with our director Joe after qualifying. Go Haz! #realestate  #f4au  #motorsport #cams https://t.co/ORvVDW7ENI"\n', 'pos'), 
         ('"The Five Most Expensive Homes for Sale in the U.S. https://t.co/Vh7S6GvYrd #5ofthebest @RWDrummoyne @RayWhiteGroup #auction #auctionsells"\n', 'neu'), 
         ('"Home and dry in time to hang up stockings https://t.co/WgDlgrOsbY"\n', 'neu'), 
         ('"59 years old.. Never used.. Could it be advertised as brand new? http://t.co/xCfWmAwYva"\n', 'pos'), 
         ('"that feeling.....\\n#lspp #pottspoint #elizabethbay #realestate #property #work #feelslikemonday #amusing #sydney https://t.co/CUMuGoVDAu"\n', 'neg'), 
         ('"Who\'s headed to dinner in Walsh Bay this evening? We highly recommend Ventuno Restaurant!\\n\\nExplore their menu -&gt;... http://t.co/XuDAHHpYtp"\n', 'pos'), 
         ('"At the epicenter of St Leonards is this perfect pad or blue chip investment. Up for auction this Saturday, 21st... http://t.co/Vm3KlSvkR1"\n', 'neu'), 
         ('"A good yarn versus the truth - Chris Wilkins - Tags: Ray White Drummoyne - Auction clearance rates - The Real Estate\\u2026https://t.co/PeqqpvJlSK"\n', 'neu'), 
         ('"Come along and see our auction today at 12.00 noon.... http://t.co/Jt1HSWGa0f"\n', 'neu'), 
         ('"Now selling by the Bay Run: 9/295 Lilyfield Road, Lilyfield\\nPreview this Saturday, 31st January from 12.30 -... http://t.co/LL78b0oezk"\n', 'neu'), 
         ('"Demand for office space in Sydney and Melbourne on the rise http://t.co/kZlylwY2He"\n', 'pos'), 
         ('"RT @BelleAnnandale: CAREER OPPORTUNITY | Sales Assistant\\n\\nAre you looking for a career in real estate? Are you ambitious with a love... htt\\u2026"\n', 'pos'), 
         ('"RT @corelogicau: The weakest performing capital city over the 3 months to March 2015 was Perth at -2.7%. http://t.co/7ZlYGbZFCO   #homevalu\\u2026"\n', 'neg'), 
         ('"2 Lion Street,Croydon just #SOLD prior to #AUCTION for an astounding $1,220,000!\\nThinking of selling? Call 9642 4288 https://t.co/O1xfIsHr7k"\n', 'pos'), 
         ('"Sure it would\\u2019ve been different if #888 racing were involved! #VASC https://t.co/CuVuO777nQ" \n', 'neg'), 
         ('"@BFerialdi But also I\'m not there tonight. I\'m setting fire to your house on account of the spider"\n', 'neg'), 
         ('"@BiggBoss i want to kick off to #swamiom  nd #PriyankaJagga because both are not deserve to stay in the house."\n', 'neg'), 
         ('"Going into any warehouse, rookie SAW mistake #jigsawmovie #JigsawSaves His game. His rules."\n', 'neu'), 
         ('"Boys Like Girls, A Rocket To The Moon, Owl City, Parachute, Lifehouse, FM Static, Fall Out Boy, Paramore, Simple Plan, Green Day, Nickelback"\n', 'neu'), 
         ('"How can western power be serious? They want to charge solar panel customers more for electricity. Encourage greenhouse reduction?"\n', 'neg'), 
         ('"Beautiful houses of Charleston South Carolina!"\n', 'pos'), 
         ('"This is a bit too cute! http://t.co/kkGurmF5YM"\n', 'pos'), 
         ('"RT @BeaufortStNet: Beaufort St Art Market Sun 9am-1pm - http://t.co/gEmCQaiVlm @tweetperth @StreetsofPerth @perthlife_ @hello_perth http://\\u2026"\n', 'neu'), 
         ('"Couldn\'t wipe the smile off his face or mine! Early morning treats.  @therestorelake #fearthebeard\\u2026 https://t.co/qxPuyaTJnU"\n', 'pos'), 
         ('"Be sure to check out my new website! Been a lot of work but over the moon with the result #realestate #hustle #perth\\nhttps://t.co/M3s8bLRjjz"\n', 'pos'), 
         ('"***UNDER OFFER***\\n3166 Albany Highway amount Nasura! \\n\\n#TheSAVVYAgent\\n\\n#property #sales #forsale\\u2026 https://t.co/dIzBjwTccf"\n', 'neu'), 
         ('"Another great day in the Perth Real Estate market. After over a month of negotiations, a lot of late nights and... https://t.co/dl6DyyF8lE"\n', 'pos'), 
         ('"Time is running out to make an offer on this beauty! #HALO #penthouse #perth #forsale #westernaustralia #cityviews https://t.co/Ng1Y0HWkoU"\n', 'pos'), 
         ('"GREAT NEWS!! First homebuyers to receive $15,000 in 2017 https://t.co/HrUwRsforv &amp; New income limits for Keystart can help you! @tweetperth"\n', 'pos'), 
         ('"When is the right time to go to market in 2016?\\n\\nSky Business interview with Real Estate Expert Tom Panos (skip... https://t.co/kuKIT0YCEC"\n', 'neu'), 
         ('"Quick breakfast in the courtyard littlewaynedlands delicious \\ud83d\\ude0b! @alexandrakate9 #nedlands\\u2026 https://t.co/OyX4b579RS"\n', 'pos'), 
         ('"Haha gold! \\ud83d\\ude0d\\ud83d\\ude0f #gymlife #cantthinkofhashtags https://t.co/twwQNETk5A"\n', 'pos'), 
         ('"Great to use a person in the real estate industry with worldly local knowloedge. - Greg Chapman agent review |... https://t.co/FVadUpuw8k"\n', 'pos'), 
         ('"I added a video to a @YouTube playlist https://t.co/PhWOs4fPiQ Be With You - Enrique Iglesias"\n', 'neu'), 
         ('"@DavidMarsh2009 @CStannage Close. Pearce will play if healthy. Tucker too."\n', 'pos'), 
         ('"**JUST LISTED**\\n\\n4x2x2 + study+theater room\\n\\nVery stylish - very modern. \\n\\n20 Ross St has it all. This is a must... https://t.co/IesSQofv5C"\n', 'neu'), 
         ('"My 15th episode of #househuntersinternational #hhi #realestate #realtor #australia #perth  https://t.co/miSY1Falxm @joshmezger"\n', 'neu'), 
         ('"28 Reasons to visit Beaufort Street, can think of loads more too!! https://t.co/ifXFplKgsM"\n', 'pos'), 
         ('"@tailopez When does this close?"\n', 'neu'), 
         ('"What successful people post in their out of office replies. \\n\\nInteresting...\\n\\nhttps://t.co/rT8kqlruem"\n', 'pos'), 
         ('"Best pest control prices for your every pest concern. https://t.co/kQzmHmFACG  #Perth #Pests #Bugs https://t.co/gNGvgHrn2V"\n', 'neu'), 
         ('"Everyone loves an Old Perth Home   (  Thanks REIWA )\\n\\nhttps://t.co/ndJar84Syu"\n', 'pos'), 
         ('"I added a video to a @YouTube playlist https://t.co/1c3HkqAXFs Nine Days - Absolutely (Story of a Girl)"\n', 'neu'), 
         ('"Serious amount of fluoro boots out on the g"\n', 'neg'), 
         ('"I added a video to a @YouTube playlist https://t.co/asxudG6z4o Ricky Martin - Livin\' La Vida Loca"\n', 'neu'), 
         ('"Contact me today for all your real estate needs \\n(613) 808-8912 \\nhttps://t.co/aK0YI70qm2\\n\\nC9PL12 HYNDMAN RD\\n1 acre\\u2026 https://t.co/aWmorPHcs7"\n', 'pos'), 
         ('"According to REIWA, as of 23/01/18, there were 9,674 properties available for rent in the Perth Metro area, down fr\\u2026 https://t.co/mgR4YtU4cQ"\n', 'neu'), 
         ('"Contact me today for all your real estate needs (613) 808-8912 \\nhttps://t.co/9pOLfpDB2u\\n\\n#Realtor\\u2026 https://t.co/scTcaNe1EN"\n', 'neu'), 
         ('"Where WA\'s 20,000 new homes are being built and why it\'s bad news https://t.co/jFUlb8h39p"\n', 'neg'), 
         ('"RT @FortescueNews: Fortescue has proudly supported Australia\\u2019s most gender-equal sports through sponsorship of the @Kookaburras and @Hockey\\u2026"\n', 'neu'), 
         ('"RT @sgtaylorkw: Large #lot with beautiful #meadow and #maturetrees along the edges.A perfect place to build your #dreamfamilyhome in the #c\\u2026"\n', 'pos'), 
         ('"Contact me today 613-808-8912 \\nConditional sale \\n$475,000 per ACRE! PRIME DEVELOPMENT SITE,3 minutes from HWY 416\\u2026 https://t.co/YbdjPlWhGt"\n', 'neu'), 
         ('"RT @pboockvar: I don\'t care if you\'re bullish, bearish or indifferent but if you\'re not in awe of what this market has done of late you hav\\u2026"\n', 'neg'), 
         ('"Would you like to join the #DiJonesFamily?\\n#DiJones #RealEstate #Recruitment #EasternSuburbs #NorthShore... https://t.co/0aycFAA6G5"\n', 'pos'), 
         ('"In 2018 When You See US #Earnings Are Up You Now Know Why - This Year\'s #Stock #Buybacks Are Already Bigger Than Al\\u2026 https://t.co/V6rdoyziEx"\n', 'pos'), 
         ('"There\'s Always Signs Of A Top In #Markets - A Few Days Ago It Was Reported The Biggest Ever Inflow Into #Equities.\\u2026 https://t.co/WcJJK4Gqza"\n', 'pos'), 
         ('"RT @RanMarineTech: #WasteShark chomping floating plastic! #Turnthetideonoplastic #plasticfree  #plasticpollution #Plasticsoup https://t.co/\\u2026"\n', 'neu'), 
         ('"Anyone Who Believes The #Market Is Not Fake &amp; Manipulated Is Crazy - How Do You Explain The DOW Jumping 500 Points\\u2026 https://t.co/fE7rQCE83e"\n', 'neg'),
         ('"@keithlaw32 Not yet we havent reached the tipping point. Watch closely the US Govt 10yr yields. Im sure if they rea\\u2026 https://t.co/wgsUWo1sC0"\n', 'neg'), 
         ('"Great start to the year by our hard working property management team!\\n #pottspoint #elizabethbay #rushcuttersbay\\u2026 https://t.co/etdUYba0cN"\n', 'pos'), 
         ('"Check out my latest review on @RateMyAgent_AUS https://t.co/3pV89PzQVy"\n', 'neu'), 
         ('"Sydney\\u2019s Kings Cross about to undergo its most profound change since the 1960\'s https://t.co/8vWF2oJ2pd https://t.co/E5qoxuOt15"\n', 'neu'), 
         ('"ABS data shows that #Sydney #housing, on the whole, gained around 70 per cent in value over the past five years\\u2026 https://t.co/2hmY9Yupqq"', 'neu')]

In [6]:
print len(train)

100


In [13]:
print train[0][1]

neg


In [19]:
#count the negative, positive and neutral data in the labelled trainning set
train_neg = 0
train_pos = 0
train_neu = 0
for line in train:
    sentiment = line[1]
    if sentiment =='neg':
        train_neg +=1
    elif sentiment =='pos':
        train_pos +=1
    elif sentiment =='neu':
        train_neu +=1
print train_neg
print train_pos
print train_neu

16
43
41


In [6]:
# Step 2
dictionary = set(word.lower() for passage in train for word in normalization(passage[0].decode('unicode_escape').encode('ascii','ignore'),stop))



set(['coach', '#earnings', 'skip', '#jigsawmovie', 'child', 'month', '#market', 'pearce', 'gt', "friend's", 'messy', '15,000', '@kookaburras', 'everything', 'electricity', 'send', 'charge', 'th', '#scenic', 'smile', 'customer', 'woman', 'street', 'gender-equal', '@2ue', 'rise', "women's", '#gc600', 'x2x2', 'kick', 'wipe', 'minute', 'leonards', 'taphouse', 'school', '@hello_perth', '9,674', 'list', 'ryan', 'try', '#thesavvyagent', 'race', '12.00', 'quick', 'havent', '295', 'keystart', '#pottspoint', 'work', 'force', 'convo', '#stock', 'chomp', 'nickelback', 'past', 'go', 'htt', 'video', '@perthlife_', 'investment', 'even', 'defence', 'power', 'kingsford', 'sun', 'bearish', '#888', 'access', '#buybacks', 'abs', 'goes', 'capital', 'international', '@therestorelake', 'ever', 'full', 'tags', 'never', "colleague's", 'youngest', 'reported', 'explore', '#balmain', 'baseball', 'along', 'teacher', 'change', '@corelogicau', 'dry', 'boy', 'great', 'daughter', 'metro', 'healthy', '31', 'qualify', '

In [8]:
# Step 3
t = [({word: (word in normalization(x[0].decode('unicode_escape').encode('ascii','ignore'),stop)) for word in dictionary}, x[1].decode('unicode_escape').encode('ascii','ignore')) for x in train]


[({'demo': False, 'coach': False, '#earnings': False, 'skip': False, '#jigsawmovie': False, 'demand': False, 'month': False, '#market': False, 'pearce': False, 'gt': False, "friend's": False, 'go': False, '15,000': False, '@kookaburras': False, 'staffy': False, 'electricity': False, 'send': False, 'charge': False, 'th': False, '#scenic': False, 'smile': False, 'customer': False, 'woman': False, 'street': False, '@2ue': False, 'rise': False, "women's": False, '#gc600': False, 'x2x2': False, 'fall': False, 'wipe': False, 'minute': False, 'leonards': False, 'taphouse': False, 'school': False, '@hello_perth': False, '9,674': False, 'list': False, 'ryan': False, 'try': False, '#thesavvyagent': False, 'race': False, 'team': False, 'quick': False, 'havent': False, '295': False, 'keystart': False, '#pottspoint': False, 'work': True, 'force': False, 'personality': False, '#stock': False, 'chomp': False, 'past': False, 'messy': False, 'htt': False, 'video': False, '@perthlife_': False, 'harder':

In [10]:
# Step 4 – the classifier is trained with sample data
classifier = nltk.NaiveBayesClassifier.train(t)


In [20]:
from sklearn.svm import LinearSVC

classifier_svm = nltk.classify.SklearnClassifier(LinearSVC())
classifier_svm.train(t)

<SklearnClassifier(LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))>

In [21]:
####### classifier labelled data
test_trained_data = []

with open(fname, 'r') as f:
    for line in f:
        test_data = line
        test_data_features = {word.lower(): (word in word_tokenize(test_data.lower())) for word in dictionary}
        emo = classifier.classify(test_data_features)
        line = (line, emo)
        test_trained_data.append(line)

print len(test_trained_data)

100


In [13]:
####### compare: classifier labelled data && manually labelled data
i = 0
correct = 0
incorrect = 0
for i in range(100):
    if train[i][1] == test_trained_data[i][1]:
        correct += 1
    else:
        incorrect += 1
    i += 1
        
print correct
print incorrect        


89
11


In [11]:
####### classifier labelled data
test_trained_data = []

with open(fname, 'r') as f:
    for line in f:
        test_data = line
        test_data_features = {word.lower(): (word in word_tokenize(test_data.lower())) for word in dictionary}
        emo = classifier_svm.classify(test_data_features)
        line = (line, emo)
        test_trained_data.append(line)

 

In [15]:
####### compare: classifier labelled data && manually labelled data
i = 0
correct = 0
incorrect = 0
for i in range(100):
    if train[i][1] == test_trained_data[i][1]:
        correct += 1
    else:
        incorrect += 1
    i += 1
        
print correct
print incorrect        


98
2


In [22]:
####### classifier labelling data
lines_count = 0
with open('sydneyrealtor2017-tweets.json', 'r') as f,open('svm-sentiment-sydneyrealtor2017-tweets.json', 'w') as b:
#with open('tweets.json', 'r') as f :
    for line in f:
        test_data = line
        test_data_features = {word.lower(): (word in normalization(test_data.decode('unicode_escape').encode('ascii','ignore'),stop)) for word in dictionary}
        emo = classifier_svm.classify(test_data_features)
        line = (line, emo)
        #test_trained_data.append(line)
        b.write(json.dumps(line))
        b.write('\n')
        
        lines_count += 1
#print lines_count